In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazon-product-length-prediction-dataset/dataset/sample_submission.csv
/kaggle/input/amazon-product-length-prediction-dataset/dataset/train.csv
/kaggle/input/amazon-product-length-prediction-dataset/dataset/test.csv


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Embedding, LSTM, Input, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [3]:
train_df = pd.read_csv('/kaggle/input/amazon-product-length-prediction-dataset/dataset/train.csv')
test_df = pd.read_csv('/kaggle/input/amazon-product-length-prediction-dataset/dataset/test.csv')


In [4]:
# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_df['TITLE'], train_df['PRODUCT_LENGTH'], test_size=0.3, random_state=42)


In [5]:
# Convert the titles to sequences of integers
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts([str(x) for x in X_train])
X_train = tokenizer.texts_to_sequences([str(x) for x in X_train])
X_val = tokenizer.texts_to_sequences([str(x) for x in X_val])

# Pad the sequences to a fixed length
max_length = 50
X_train = pad_sequences(X_train, maxlen=max_length, padding='post', truncating='post')
X_val = pad_sequences(X_val, maxlen=max_length, padding='post', truncating='post')

# Define the model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=32, input_length=max_length))
model.add(LSTM(units=32))
model.add(Dense(units=1, activation='linear'))

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_absolute_error')

# Train the model
model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model on the validation set
score = model.evaluate(X_val, y_val, verbose=0)
print("Mean absolute error on validation set:", score)

Epoch 1/10
49213/49213 [==============================] - 470s 9ms/step - loss: 3899.3774 - val_loss: 2973.0220
Epoch 2/10
49213/49213 [==============================] - 427s 9ms/step - loss: 3746.7666 - val_loss: 2936.4260
Epoch 3/10
49213/49213 [==============================] - 435s 9ms/step - loss: 3720.4790 - val_loss: 2919.7666
Epoch 4/10
49213/49213 [==============================] - 430s 9ms/step - loss: 3703.7656 - val_loss: 2906.6284
Epoch 5/10
49213/49213 [==============================] - 433s 9ms/step - loss: 3689.5017 - val_loss: 2895.8367
Epoch 6/10
49213/49213 [==============================] - 436s 9ms/step - loss: 3678.4131 - val_loss: 2888.1663
Epoch 7/10
49213/49213 [==============================] - 436s 9ms/step - loss: 3668.2166 - val_loss: 2880.1763
Epoch 8/10
49213/49213 [==============================] - 437s 9ms/step - loss: 3659.1851 - val_loss: 2873.6855
Epoch 9/10
49213/49213 [==============================] - 428s 9ms/step - loss: 3650.5830 - val_loss: 28

In [9]:
print("stmt 7")
new_titles = test_df['TITLE'].astype(str)
new_seqs = tokenizer.texts_to_sequences(new_titles)
new_padded_seqs = pad_sequences(new_seqs, maxlen=max_length, padding='post', truncating='post')
predictions = model.predict(new_padded_seqs)
predictions = np.array(predictions).flatten()
output = pd.DataFrame({'PRODUCT_ID' : test_df['PRODUCT_ID'], 'PRODUCT_LENGTH' : predictions})
output.to_csv('submission.csv', index = False)

stmt 7
22961/22961 [==============================] - 70s 3ms/step


In [19]:
# X_test = tokenizer.texts_to_sequences(test_df['TITLE'])
# X_test = pad_sequences(X_test, maxlen=max_length, padding='post', truncating='post')
# id_test = test_df['PRODUCT_ID'].astype(str) # convert to string
# #y_pred = model.predict([X_test, id_test])
# import tensorflow as tf
# tf.config.run_functions_eagerly(True)

# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# y_pred = model.predict([X_test, id_test])

X_test = tokenizer.texts_to_sequences(test_df['TITLE'])
X_test = pad_sequences(X_test, maxlen=max_length, padding='post', truncating='post')
y_pred = model.predict(X_test)


ValueError: Unexpected result of `predict_function` (Empty batch_outputs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.

In [ ]:
# Save the predictions to a CSV file
output_df = pd.DataFrame({'PRODUCT_ID': id_test, 'PREDICTED_LENGTH': y_pred.flatten()})
output_df.to_csv('predictions.csv', index=False)